## Import libraries

In [1]:
import sys
import requests
from haqs_api import haqs_api

NoneType = type(None)

%load_ext autoreload
%autoreload 2

## Data addresses

1. **Stations request** http://api.gios.gov.pl/pjp-api/rest/station/findAll
2. **Sensors request** http://api.gios.gov.pl/pjp-api/rest/station/sensors/{stationId}
3. **Data request** http://api.gios.gov.pl/pjp-api/rest/data/getData/{sensorId}
4. **AQ index request** http://api.gios.gov.pl/pjp-api/rest/aqindex/getIndex/{stationId}

# Geopandas part

## Get Stations from GIOŚ API

In [2]:
stations_json = haqs_api.get_stations()
print("Number of available stations: ", len(stations_json))

Number of available stations:  158


In [3]:
stations_df = haqs_api.create_stations_gdf(stations_json)
stations_df.head()

,station_id,geometry
0,114,POINT (17.141125 51.115933)
1,117,POINT (17.02925 51.129378)
2,129,POINT (17.012689 51.086225)
3,52,POINT (16.180513 51.204503)
4,109,POINT (16.269677 50.768729)


## Plot Stations on map

In [7]:
haqs_api.create_stations_map(stations_json)

## Get list of Sensors for each Station

In [5]:
sensors_df = haqs_api.create_sensors_df(stations_json)
sensors_df.head()

,station_id,sensor_id,parameter
0,114,642,NO2
1,114,644,O3
2,117,660,CO
3,117,14395,PM10
4,117,658,C6H6


## Get Sensors readings from GIOŚ API

In [6]:
sensors_df = haqs_api.get_latest_sensors_readings(sensors_df)
sensors_df.head()

,station_id,sensor_id,parameter,data
0,114,642,NO2,36.74230
1,114,644,O3,7.11056
2,117,660,CO,602.70600
3,117,14395,PM10,18.71650
4,117,658,C6H6,0.15146


In [7]:
available_parameters = haqs_api.get_available_parameters(sensors_df)
available_parameters

['NO2', 'O3', 'CO', 'PM10', 'C6H6', 'PM2.5', 'SO2']

## Visualize readings

In [8]:
params_df = haqs_api.get_param_df(stations_df, sensors_df, 'PM10')
haqs_api.show_readings_map(params_df)

Loading BokehJS ...

# Connect with database

In [2]:
conn = haqs_api.connect_with_db()

Successfully connected with DataBase!


In [ ]:
haqs_api.close_db_connection(conn)

## Create PostGIS Extension

In [10]:
haqs_api.create_postgis_extension(conn)

extension "postgis" already exists



## <font color="blue">Station Table</font>

### Create Stations Table

In [11]:
haqs_api.create_stations_table(conn)

relation "stations" already exists



### Insert Stations into DataBase

In [3]:
stations_json = haqs_api.get_stations()

In [4]:
stations_json[0]

{'id': 114,
 'stationName': 'Wrocław - Bartnicza',
 'gegrLat': '51.115933',
 'gegrLon': '17.141125',
 'city': {'id': 1064,
  'name': 'Wrocław',
  'commune': {'communeName': 'Wrocław',
   'districtName': 'Wrocław',
   'provinceName': 'DOLNOŚLĄSKIE'}},
 'addressStreet': 'ul. Bartnicza'}

In [ ]:
# iterate over stations
for station in stations_json:
    
    station_id = station["id"]
    station_lon = station['gegrLon']
    station_lat = station['gegrLat']
    
    try:
        haqs_api.db_insert_station(conn, station_id, station_lon, station_lat)
    except Exception as e:
        print(e)
        print ("Cannot execute insertion for Station ID: {}".format(station_id))

### Show Insertions

In [16]:
haqs_api.show_insertions(conn, "stations").head()

,station_id,geom
0,114,0101000020E6100000E3A59BC4202431407AC37DE4D68E...
1,117,0101000020E6100000736891ED7C073140346953758F90...
2,129,0101000020E6100000DA0418963F0331407E8CB96B098B...
3,52,0101000020E6100000BB809719362E30408A7780272D9A...
4,109,0101000020E6100000C07B478D094530404F3E3DB66562...


### Create Stations DataFrame from Stations Table

In [8]:
stations_gdf = haqs_api.return_stations_gdf(conn)
stations_gdf.head()

,station_id,geom
0,114,POINT (17.141125 51.115933)
1,117,POINT (17.02925 51.129378)
2,129,POINT (17.012689 51.086225)
3,52,POINT (16.180513 51.204503)
4,109,POINT (16.269677 50.768729)


## <font color="blue">Sensors Table</font>

### Create Sensors Table

In [17]:
haqs_api.create_sensors_table(conn)

relation "sensors" already exists



### Insert Sensors into DataBase

In [18]:
# iterate over stations
for station in stations_json:
    
    station_id = station["id"]
    station_lon = station['gegrLon']
    station_lat = station['gegrLat']
    
    sensors_json = requests.get("http://api.gios.gov.pl/pjp-api/rest/station/sensors/{}".format(station_id)).json()
    
    # iterate over station sensors
    for sensor in sensors_json:
        
        sensor_id = sensor["id"]
        sensor_parameter = sensor["param"]["paramCode"]
    
        try:
            haqs_api.db_insert_sensor(conn, sensor_id, sensor_parameter, station_id)
        except Exception as e:
            print(e)
            print ("Cannot execute insertion for Sensor ID: {} (Station ID: {})".format(sensor_id, station_id))

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(642) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(644) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(660) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(14395) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(658) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(665) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(670) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(667) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(672) already exists.

duplicat

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(2370) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(2377) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(2373) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(2378) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(2382) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(16228) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(16250) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(16249) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(16231) already exists

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(5716) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(5721) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(5718) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(5720) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(5722) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(16424) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(5826) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(16396) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(16173) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(5343) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(5346) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(5354) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(5376) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(5371) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(5378) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(5373) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(5382) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(5480) already exists.

d

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(17249) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(18012) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(18145) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(19816) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(18069) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(17326) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(17800) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(17796) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(17799) already e

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(4701) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(4706) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(4703) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(4707) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(4709) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(4715) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(4712) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(4714) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(4717) already exists.

d

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(14804) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(14826) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(16486) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(16488) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(16498) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(16905) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(16904) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(3576) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(3584) already exi

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(18009) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(15727) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(1676) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(14348) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(1741) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(1983) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(16744) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(14367) already exists.

duplicate key value violates unique constraint "sensors_pkey"
DETAIL:  Key (sensor_id)=(2001) already exist

### Create Sensors DataFrame and GeoDataFrame from Sensors Table

#### DataFrame

In [5]:
sensors_df = haqs_api.return_sensors_df(conn)
sensors_df.head()

,sensor_id,sensor_parameter,station_id
0,642,NO2,114
1,644,O3,114
2,660,CO,117
3,14395,PM10,117
4,658,C6H6,117


#### GeoDataFrame

In [6]:
sensors_gdf = haqs_api.return_sensors_gdf(conn)
sensors_gdf.head()

,sensor_id,sensor_parameter,station_id,geom
0,642,NO2,114,POINT (17.141125 51.115933)
1,644,O3,114,POINT (17.141125 51.115933)
2,660,CO,117,POINT (17.02925 51.129378)
3,14395,PM10,117,POINT (17.02925 51.129378)
4,658,C6H6,117,POINT (17.02925 51.129378)


## <font color="blue">Readings Table</font>

### Create Readings Table  

In [20]:
haqs_api.create_readings_table(conn)

relation "readings" already exists



### Create list of available Sensors

In [8]:
sensors_ids = haqs_api.return_sensors_ids(conn)
sensors_ids[:2]

[642, 644]

### Insert Readings Into DataBase

#### Insert all available non NULL readings into Readings Table

In [10]:
for sensor_id in sensors_ids:

    data_json = requests.get("http://api.gios.gov.pl/pjp-api/rest/data/getData/{}".format(sensor_id)).json()
    data = data_json['values']
    
    # check all available data
    for row in data:

        date = row['date']
        reading = row['value']

        # if there is no value wait until next attempt
        if isinstance(reading, NoneType):
            continue

        try:
            haqs_api.db_insert_sensor_readings(conn, sensor_id, date, reading, date, sensor_id)
        except Exception as e:
            print(e)

#### Insert latest non NULL readings into Readings Table

In [9]:
for sensor_id in sensors_ids:

    data_json = requests.get("http://api.gios.gov.pl/pjp-api/rest/data/getData/{}".format(sensor_id)).json()
    data = data_json['values']
    
    # check latest available data
    for row in data[:1]:

        date = row['date']
        reading = row['value']

        # if there is no value wait until next attempt
        if isinstance(reading, NoneType):
            continue

        try:
            haqs_api.db_insert_sensor_readings(conn, sensor_id, date, reading, date, sensor_id)
        except Exception as e:
            print(e)

### Create Readings DataFrame from Readings Table

#### DataFrame

In [23]:
readings_df = haqs_api.return_readings_df(conn)
readings_df.head()

,sensor_id,date,reading
0,642,2018-09-07 18:00:00,12.93730
1,642,2018-09-07 17:00:00,12.83960
2,642,2018-09-07 16:00:00,11.44120
3,642,2018-09-07 15:00:00,9.43838
4,642,2018-09-07 14:00:00,10.22020


In [24]:
readings_df.describe()

,sensor_id,reading
count,118371.000000,118371.000000
mean,6990.866775,61.487632
std,5994.143952,131.421811
min,50.000000,0.000000
25%,2519.000000,4.264930
50%,4819.000000,15.599500
75%,14618.000000,39.720950
max,19816.000000,2928.870000


In [25]:
readings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118371 entries, 0 to 118370
Data columns (total 3 columns):
sensor_id    118371 non-null int64
date         118371 non-null object
reading      118371 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 2.7+ MB


#### GeoDataFrame

In [31]:
readings_gdf = haqs_api.return_readings_gdf(conn, limit=20)
print("GeoDataFrame length: ",len(readings_df))
readings_gdf.head()

DataFrame length:  20


,sensor_id,date,reading,sensor_parameter,station_id,geom
0,293,2018-09-17 18:00:00,121.98900,O3,52,POINT (16.180513 51.204503)
1,616,2018-09-17 18:00:00,107.04800,O3,109,POINT (16.269677 50.768729)
2,285,2018-09-17 18:00:00,293.50600,CO,52,POINT (16.180513 51.204503)
3,291,2018-09-17 18:00:00,9.30772,NO2,52,POINT (16.180513 51.204503)
4,608,2018-09-17 18:00:00,285.88500,CO,109,POINT (16.269677 50.768729)


In [51]:
pm10_gdf = haqs_api.return_parameter_gdf(conn, parameter='PM10')
pm10_gdf.head()

,sensor_id,date,reading,sensor_parameter,station_id,geom
0,965,2018-09-17 19:00:00,45.9,PM10,156,POINT (17.987906 53.121764)
1,995,2018-09-17 19:00:00,39.1,PM10,158,POINT (17.995708 53.134083)
2,2069,2018-09-17 19:00:00,33.0,PM10,295,POINT (19.529786 51.75805)
3,2199,2018-09-17 19:00:00,23.0,PM10,314,POINT (19.368683 51.667981)
4,2377,2018-09-17 19:00:00,28.0,PM10,355,POINT (19.421231 51.856692)


## Create and save GeoDataFrame to Shapefile

In [57]:
stations_df.head()

,station_id,geom
0,114,POINT (17.141125 51.115933)
1,117,POINT (17.02925 51.129378)
2,129,POINT (17.012689 51.086225)
3,52,POINT (16.180513 51.204503)
4,109,POINT (16.269677 50.768729)


In [58]:
sensors_df.head()

,sensor_id,sensor_parameter,station_id
0,642,NO2,114
1,644,O3,114
2,660,CO,117
3,14395,PM10,117
4,658,C6H6,117


In [59]:
readings_df.head()

,sensor_id,date,reading
0,642,2018-09-07 18:00:00,12.93730
1,642,2018-09-07 17:00:00,12.83960
2,642,2018-09-07 16:00:00,11.44120
3,642,2018-09-07 15:00:00,9.43838
4,642,2018-09-07 14:00:00,10.22020


In [60]:
sensors_df = sensors_df.merge(stations_df, on='station_id')
sensors_df.head()

,sensor_id,sensor_parameter,station_id,geom
0,642,NO2,114,POINT (17.141125 51.115933)
1,644,O3,114,POINT (17.141125 51.115933)
2,660,CO,117,POINT (17.02925 51.129378)
3,14395,PM10,117,POINT (17.02925 51.129378)
4,658,C6H6,117,POINT (17.02925 51.129378)


In [61]:
readings_df = gpd.GeoDataFrame(readings_df.merge(sensors_df, on='sensor_id'))
readings_df.crs = from_epsg(4326)
readings_df.rename(index=str, columns={"geom": "geometry"}, inplace=True)
readings_df.head()

,sensor_id,date,reading,sensor_parameter,station_id,geometry
0,642,2018-09-07 18:00:00,12.93730,NO2,114,POINT (17.141125 51.115933)
1,642,2018-09-07 17:00:00,12.83960,NO2,114,POINT (17.141125 51.115933)
2,642,2018-09-07 16:00:00,11.44120,NO2,114,POINT (17.141125 51.115933)
3,642,2018-09-07 15:00:00,9.43838,NO2,114,POINT (17.141125 51.115933)
4,642,2018-09-07 14:00:00,10.22020,NO2,114,POINT (17.141125 51.115933)


In [62]:
readings_df.to_file('shp/readings.shp')